In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from catboost import CatBoostClassifier, Pool
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.metrics import plot_confusion_matrix, roc_auc_score, plot_roc_curve, classification_report
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
test  = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')

**The Purpose of this Notebook is to use the data directly and train on it and do prediction. Usually if we have a mix of Categorical and Continuous Features we need to convert it to ONE HOT Encoding. But Boosting frameworks now provide a way to directly use Categorical Features. **

In [ ]:
print(f'train: {len(train)}, test: { len(test)}')
train.head()


In [ ]:
x_train = train.drop(columns=['id', 'target'])
y_train = train['target']
x_train = x_train.values
y_train = y_train.values

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Need to specify which columns are Categorical. You need to provide the indices of columns and also the name of the columns.
#categorical_features = [col for c, col in enumerate(train.columns) if 'cat' in col]
categorical_features =[0,1,2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18]

In [ ]:
model_1 = CatBoostClassifier(verbose=True)
#model_1.fit(x_train, y_train, cat_features=categorical_features)

**The Model with CatBoost Got the score 0.77076 on Public LB. As it is very naive model, so it is expected.**

*Now we can test the new Framework lightGBM *

In [ ]:
obj_feat_train = list(train.loc[:, train.dtypes == 'object'].columns.values)
obj_feat_test = list(test.loc[:, test.dtypes == 'object'].columns.values)
for feature in obj_feat_train:
    train[feature] = train[feature].astype('category')
for feature in obj_feat_test:
    test[feature] = test[feature].astype('category')
    

x_train = train.drop(columns=['id', 'target'])
y_train = train['target']

x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
import lightgbm as lgb
train_data = lgb.Dataset(x_train, label=y_train,categorical_feature=[0,1,2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18])
valid_data = lgb.Dataset(x_valid, label=y_valid,categorical_feature=[0,1,2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18])

#Specifying the parameter
params={}
params['objective']='binary' #Binary target feature
params['metric']='binary_logloss' #metric for binary classification
#train the model 
#model_2 = lgb.train(params,train_data, valid_sets=valid_data) #0.347189

**LightGBM increase the score to 0.88537, which is a lot without any parameter tuning. CatBoost can also get this score but it need to be tuned.**
> VERSION: 03 **Now we will do some tuning this lightGBM to make it work more better.** 

A person in the comments suggested me to try the OPtuna OPtimization. SO i thought why not we give it a try so here. we are trying that.

[Thanks to Raj Gandhi for K_FOLD_CV Function](https://www.kaggle.com/rajgandhi/tps-march-lgbm-7-fold-cv)

In [ ]:
def K_fold_CV(X, y, model, params, folds=5):
    roc_score = []
    # preds = np.zeros(len(test))
    #        yp += model.predict_proba(X_test)[:, 1] / k
    # Using Stratified K-fold CV for preserving the percentage of samples for each classes
    skf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=42)
    for fold, (tr_idx, ts_idx) in enumerate(skf.split(X, y)):
        print(f"Fold: {fold}")
        x_tr, y_tr = X.iloc[tr_idx], y.iloc[tr_idx]
        x_ts, y_ts = X.iloc[ts_idx], y.iloc[ts_idx]

        clf = model(**params)
        clf.fit(x_tr, y_tr,
                eval_set=[(x_ts, y_ts)],
                early_stopping_rounds=100,
                verbose=False)

        score = roc_auc_score(y_ts, clf.predict_proba(x_ts)[:, 1])
        roc_score.append(score)
        print(f"ROC AUC Score: {score}")
        print("-"*25)
    
    return clf, np.mean(roc_score)

In [ ]:
import optuna
import sklearn
# 1. Define an objective function to be maximized.
# def objective(trial):
    
#     train_data = lgb.Dataset(x_train, label=y_train, categorical_feature=[0,1,2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18], free_raw_data=False)
#     valid_data = lgb.Dataset(x_valid, label=y_valid, categorical_feature=[0,1,2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18], free_raw_data=False)

#     # 2. Suggest values of the hyperparameters using a trial object.
#     param = {
#         'objective': 'binary',
#         'metric': 'binary_logloss',
#         #'boosting_type': 'dart',
#         "metric":"auc",
#         'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 10.0),
#         'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 10.0),
#         'num_leaves': trial.suggest_int('num_leaves', 2, 256),
#         'n_estimators': trial.suggest_int('n_estimators', 20, 1000),
#         'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
#         'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
#         'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
#         'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
#         'feature_pre_filter': False
#     }

#     gbm = lgb.train(param, train_data)
#     preds = gbm.predict(x_valid)
#     pred_labels = np.rint(preds)
#     accuracy = sklearn.metrics.accuracy_score(y_valid, pred_labels)
#     return accuracy

# # 3. Create a study object and optimize the objective function.
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100)
# print("Number of finished trials: {}".format(len(study.trials)))

# print("Best trial:")
# trial = study.best_trial

# print("  Value: {}".format(trial.value))

# print("  Params: ")
# for key, value in trial.params.items():
#     print("    {}: {}".format(key, value))

### Trial with gbdt: 0.84461
### Trial With dart: 0.84451

In [ ]:
obj_feat_train = list(train.loc[:, train.dtypes == 'object'].columns.values)
obj_feat_test = list(test.loc[:, test.dtypes == 'object'].columns.values)
for feature in obj_feat_train:
    train[feature] = train[feature].astype('category')
for feature in obj_feat_test:
    test[feature] = test[feature].astype('category')
    
x = train.drop(columns=['id', 'target'])
y = train['target']
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
import lightgbm as lgb
from  lightgbm import LGBMClassifier
train_data = lgb.Dataset(x_train, label=y_train,categorical_feature=[0,1,2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18])
valid_data = lgb.Dataset(x_valid, label=y_valid,categorical_feature=[0,1,2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18])

#Specifying the parameter
# Parameters get from optune Optimization I comment the code bcz it take time to run.
params = { 'objective': 'binary',
        'metric': 'binary_logloss',
        'lambda_l1': 4.778750845902905,
    'lambda_l2': 2.6003948800594086e-07,
    'num_leaves': 41,
    'n_estimators': 519,
    "metric":"auc",
    'feature_fraction': 0.45127412515749443,
    'bagging_fraction': 0.9793967272010233,
    'bagging_freq': 4,
    'min_child_samples': 94}
#train the model 
#model_optune = lgb.train(params,train_data, valid_sets=valid_data)
clf, score = K_fold_CV(x,y, LGBMClassifier, params, 7)

In [ ]:
submission = test
ids = submission['id'].values
submission.drop('id', inplace=True, axis=1)

x = submission
y = clf.predict_proba(submission[x.columns])[:, 1]#clf.predict(x)

output = pd.DataFrame({'id': ids, 'target': y})
output.to_csv("submission.csv", index=False) 